This notebook is for preparing all datasets for integration. 

This involves:
* reading in each dataset
* check metadata all correct
* add additional metadata regarding site and cancer_subtype
* add metadata for sample_type_major
* add metadata for integration_id --> samples that are not biologically distinct (eg. two biopsies from one tumour) get same id
* use integration id to merge layers --> layers in dataset will represent how they will be integrated 
* exclude any samples with <100 myeloid cells
* record number of cells

Backing up to rdm: 
``` bash
rsync -azvhp /scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/ /QRISdata/Q5935/nikita/scdata/Myeloid_Cells/Myeloid_Cells_Integrate
```

In [1]:
#set wd
getwd()
setwd('/scratch/user/s4436039/scdata/Myeloid_Cells')
getwd()

[1] "/scratch/user/s4436039/Nikita-project"

[1] "/scratch/user/s4436039/scdata/Myeloid_Cells"

In [2]:
#Load packages
library(dplyr)
library(Seurat)
library(patchwork)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




## GSE184880

In [41]:
HGSOC <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE184880_myeloid.RDS")

In [42]:
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)

An object of class Seurat 
27984 features across 7799 samples within 1 assay 
Active assay: RNA (27984 features, 2000 variable features)
 25 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, counts.7, counts.8, counts.9, counts.10, counts.11, counts.12, data.1, data.2, data.3, data.4, data.5, data.6, data.7, data.8, data.9, data.10, data.11, data.12, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE184880"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GSE184880_Cancer1_AAACCCACAGCTGCCA-1,GSE184880,9374,2655,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,15.980371,1,1
GSE184880_Cancer1_AAACCCACATGACGGA-1,GSE184880,2659,1246,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,8.837909,1,1
GSE184880_Cancer1_AAACGAACAGTAGTGG-1,GSE184880,3020,1206,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,13.807947,1,1
GSE184880_Cancer1_AAACGAATCACCCTCA-1,GSE184880,50940,6660,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,8.531606,1,1
GSE184880_Cancer1_AAACGCTTCTCCACTG-1,GSE184880,10129,2880,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,11.225195,1,1
GSE184880_Cancer1_AAACGCTTCTGCTCTG-1,GSE184880,12756,3352,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,9.321104,1,1


In [43]:
table(HGSOC$sample_type)
table(HGSOC$cancer_type)
table(HGSOC$patient_id)
table(HGSOC$sample_id)


Healthy_ovary        tumour 
         1457          6342 


Healthy   HGSOC 
   1457    6342 


Cancer1 Cancer2 Cancer3 Cancer4 Cancer5 Cancer6 Cancer7   Norm1   Norm2   Norm3 
   2298    1080     577     792     695     652     248      54     281     360 
  Norm4   Norm5 
    193     569 


GSE184880_Healthy_Norm1 GSE184880_Healthy_Norm2 GSE184880_Healthy_Norm3 
                     54                     281                     360 
GSE184880_Healthy_Norm4 GSE184880_Healthy_Norm5 GSE184880_HGSOC_Cancer1 
                    193                     569                    2298 
GSE184880_HGSOC_Cancer2 GSE184880_HGSOC_Cancer3 GSE184880_HGSOC_Cancer4 
                   1080                     577                     792 
GSE184880_HGSOC_Cancer5 GSE184880_HGSOC_Cancer6 GSE184880_HGSOC_Cancer7 
                    695                     652                     248 

In [44]:
#set site metadata
HGSOC@meta.data$site <- "ovary"

In [45]:
#set subtype metadata

#split by cancer_type
HGSOC_tumour <- subset(HGSOC, subset = cancer_type %in% c("HGSOC"))
HGSOC_healthy <- subset(HGSOC, subset = cancer_type %in% c("Healthy"))

HGSOC_tumour@meta.data$cancer_subtype <- "HGSOC"
HGSOC_healthy@meta.data$cancer_subtype <- "NA"

HGSOC_tumour@meta.data$sample_type_major <- "primary tumour"
HGSOC_healthy@meta.data$sample_type_major <- "healthy"

#Merge seurat objects back together
HGSOC <- merge(HGSOC_tumour, y = c(HGSOC_healthy), project = "GSE184880")

In [46]:
#set integration_id metadata
HGSOC@meta.data$integration_id <- HGSOC@meta.data$sample_id

In [47]:
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)

An object of class Seurat 
27984 features across 7799 samples within 1 assay 
Active assay: RNA (27984 features, 2000 variable features)
 26 layers present: counts.1.1, counts.10.2, counts.11.2, counts.12.2, counts.2.1, counts.3.1, counts.4.1, counts.5.1, counts.6.1, counts.7.1, data.1.1, data.2.1, data.3.1, data.4.1, data.5.1, data.6.1, data.7.1, scale.data.1, counts.8.2, counts.9.2, data.8.2, data.9.2, data.10.2, data.11.2, data.12.2, scale.data.2

[1] "GSE184880"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,cancer_subtype,sample_type_major,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE184880_Cancer1_AAACCCACAGCTGCCA-1,GSE184880,9374,2655,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,15.980371,1,1,ovary,HGSOC,primary tumour,GSE184880_HGSOC_Cancer1
GSE184880_Cancer1_AAACCCACATGACGGA-1,GSE184880,2659,1246,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,8.837909,1,1,ovary,HGSOC,primary tumour,GSE184880_HGSOC_Cancer1
GSE184880_Cancer1_AAACGAACAGTAGTGG-1,GSE184880,3020,1206,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,13.807947,1,1,ovary,HGSOC,primary tumour,GSE184880_HGSOC_Cancer1
GSE184880_Cancer1_AAACGAATCACCCTCA-1,GSE184880,50940,6660,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,8.531606,1,1,ovary,HGSOC,primary tumour,GSE184880_HGSOC_Cancer1
GSE184880_Cancer1_AAACGCTTCTCCACTG-1,GSE184880,10129,2880,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,11.225195,1,1,ovary,HGSOC,primary tumour,GSE184880_HGSOC_Cancer1
GSE184880_Cancer1_AAACGCTTCTGCTCTG-1,GSE184880,12756,3352,tumour,HGSOC,Cancer1,GSE184880_HGSOC_Cancer1,9.321104,1,1,ovary,HGSOC,primary tumour,GSE184880_HGSOC_Cancer1


In [48]:
#exclude any samples with <100 cells
table(HGSOC$integration_id)
#exclude Norm1
HGSOC <- subset(HGSOC, !(subset = integration_id %in% c("GSE184880_Healthy_Norm1")))
table(HGSOC$integration_id)


GSE184880_Healthy_Norm1 GSE184880_Healthy_Norm2 GSE184880_Healthy_Norm3 
                     54                     281                     360 
GSE184880_Healthy_Norm4 GSE184880_Healthy_Norm5 GSE184880_HGSOC_Cancer1 
                    193                     569                    2298 
GSE184880_HGSOC_Cancer2 GSE184880_HGSOC_Cancer3 GSE184880_HGSOC_Cancer4 
                   1080                     577                     792 
GSE184880_HGSOC_Cancer5 GSE184880_HGSOC_Cancer6 GSE184880_HGSOC_Cancer7 
                    695                     652                     248 


GSE184880_Healthy_Norm2 GSE184880_Healthy_Norm3 GSE184880_Healthy_Norm4 
                    281                     360                     193 
GSE184880_Healthy_Norm5 GSE184880_HGSOC_Cancer1 GSE184880_HGSOC_Cancer2 
                    569                    2298                    1080 
GSE184880_HGSOC_Cancer3 GSE184880_HGSOC_Cancer4 GSE184880_HGSOC_Cancer5 
                    577                     792                     695 
GSE184880_HGSOC_Cancer6 GSE184880_HGSOC_Cancer7 
                    652                     248 

In [49]:
#join layers and then split them by integration_id
Layers(HGSOC[["RNA"]])
#join layers
HGSOC[["RNA"]] <- JoinLayers(HGSOC[["RNA"]])
Layers(HGSOC[["RNA"]])
#split layers
HGSOC[["RNA"]] <- split(HGSOC[["RNA"]], f = HGSOC$integration_id)
Layers(HGSOC[["RNA"]])


[1] "counts.1.1"   "counts.10.2"  "counts.11.2"  "counts.12.2"  "counts.2.1"  
 [6] "counts.3.1"   "counts.4.1"   "counts.5.1"   "counts.6.1"   "counts.7.1"  
[11] "data.1.1"     "data.2.1"     "data.3.1"     "data.4.1"     "data.5.1"    
[16] "data.6.1"     "data.7.1"     "scale.data.1" "counts.9.2"   "data.9.2"    
[21] "data.10.2"    "data.11.2"    "data.12.2"    "scale.data.2"

[1] "scale.data" "data"       "counts"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "counts.GSE184880_HGSOC_Cancer1" "counts.GSE184880_HGSOC_Cancer2"
 [3] "counts.GSE184880_HGSOC_Cancer3" "counts.GSE184880_HGSOC_Cancer4"
 [5] "counts.GSE184880_HGSOC_Cancer5" "counts.GSE184880_HGSOC_Cancer6"
 [7] "counts.GSE184880_HGSOC_Cancer7" "counts.GSE184880_Healthy_Norm2"
 [9] "counts.GSE184880_Healthy_Norm3" "counts.GSE184880_Healthy_Norm4"
[11] "counts.GSE184880_Healthy_Norm5" "scale.data"                    
[13] "data.GSE184880_HGSOC_Cancer1"   "data.GSE184880_HGSOC_Cancer2"  
[15] "data.GSE184880_HGSOC_Cancer3"   "data.GSE184880_HGSOC_Cancer4"  
[17] "data.GSE184880_HGSOC_Cancer5"   "data.GSE184880_HGSOC_Cancer6"  
[19] "data.GSE184880_HGSOC_Cancer7"   "data.GSE184880_Healthy_Norm2"  
[21] "data.GSE184880_Healthy_Norm3"   "data.GSE184880_Healthy_Norm4"  
[23] "data.GSE184880_Healthy_Norm5"

In [50]:
#record number of cells
table(HGSOC$integration_id)


GSE184880_Healthy_Norm2 GSE184880_Healthy_Norm3 GSE184880_Healthy_Norm4 
                    281                     360                     193 
GSE184880_Healthy_Norm5 GSE184880_HGSOC_Cancer1 GSE184880_HGSOC_Cancer2 
                    569                    2298                    1080 
GSE184880_HGSOC_Cancer3 GSE184880_HGSOC_Cancer4 GSE184880_HGSOC_Cancer5 
                    577                     792                     695 
GSE184880_HGSOC_Cancer6 GSE184880_HGSOC_Cancer7 
                    652                     248 

In [51]:
#re-export seurat object ready for integration
saveRDS(HGSOC, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/GSE184880_myeloid_int.RDS")

In [52]:
#remove all objects in R
rm(list = ls())

## GSE213243

In [53]:
HGSOC_tu <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE213243_Tumour_myeloid.RDS")
HGSOC_As <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE213243_Ascites_myeloid.RDS")

In [54]:
HGSOC_tu
HGSOC_tu@project.name
head(HGSOC_tu@meta.data)

HGSOC_As
HGSOC_As@project.name
head(HGSOC_As@meta.data)

An object of class Seurat 
58825 features across 804 samples within 1 assay 
Active assay: RNA (58825 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE213243"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GSE213243_tumour_AAAGGTACACGCAGTC-1,GSE213243,8050,2780,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,19.962733,3,3
GSE213243_tumour_AAATGGACACACGCCA-1,GSE213243,5854,2467,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,4.936795,3,3
GSE213243_tumour_AACAAAGCAATTTCCT-1,GSE213243,6073,2541,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,6.323069,3,3
GSE213243_tumour_AACACACGTAGCTTTG-1,GSE213243,13497,3862,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,5.319701,3,3
GSE213243_tumour_AACACACTCGCTGTTC-1,GSE213243,8644,3306,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,10.596946,3,3
GSE213243_tumour_AACAGGGCAACCCTAA-1,GSE213243,6263,2562,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,3.544627,3,3


An object of class Seurat 
58825 features across 2688 samples within 1 assay 
Active assay: RNA (58825 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE213243"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GSE213243_ascites_AAACCCAAGTAGCAAT-2,GSE213243,16943,4684,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,7.572449,5,5
GSE213243_ascites_AAACCCACAGTCGTTA-2,GSE213243,14219,3822,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,5.021450,1,1
GSE213243_ascites_AAACCCATCCGTAGTA-2,GSE213243,15634,4224,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,6.556224,5,5
GSE213243_ascites_AAACGAAAGTGCTCGC-2,GSE213243,3007,1377,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,28.766212,6,6
GSE213243_ascites_AAACGAAGTATGGTAA-2,GSE213243,13828,4227,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,4.122071,5,5
GSE213243_ascites_AAACGCTAGTATCTGC-2,GSE213243,12945,3944,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,8.937814,6,6


In [55]:
table(HGSOC_tu$sample_type)
table(HGSOC_tu$cancer_type)
table(HGSOC_tu$patient_id)
table(HGSOC_tu$sample_id)

table(HGSOC_As$sample_type)
table(HGSOC_As$cancer_type)
table(HGSOC_As$patient_id)
table(HGSOC_As$sample_id)


tumour 
   804 


HGSOC 
  804 


pt-1 
 804 


GSE213243_HGSOC_tumour 
                   804 


ascites 
   2688 


HGSOC 
 2688 


pt-1 
2688 


GSE213243_HGSOC_ascites 
                   2688 

In [56]:
#set site metadata
HGSOC_tu@meta.data$site <- "ovary"
HGSOC_As@meta.data$site <- "ascites fluid"

HGSOC_tu@meta.data$sample_type_major <- "primary tumour"
HGSOC_As@meta.data$sample_type_major <- "ascites"

In [57]:
#set subtype metadata

#split by cancer_type
HGSOC_tu@meta.data$cancer_subtype <- "HGSOC"
HGSOC_As@meta.data$cancer_subtype <- "HGSOC"

In [58]:
#merge objects
HGSOC <- merge(HGSOC_tu, y = c(HGSOC_As), project = "GSE213243")

In [59]:
#set integration_id metadata
HGSOC@meta.data$integration_id <- HGSOC@meta.data$sample_id

In [60]:
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)
tail(HGSOC@meta.data)

An object of class Seurat 
58825 features across 3492 samples within 1 assay 
Active assay: RNA (58825 features, 2000 variable features)
 6 layers present: counts.1, counts.2, data.1, scale.data.1, data.2, scale.data.2

[1] "GSE213243"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.5,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE213243_tumour_AAAGGTACACGCAGTC-1,GSE213243,8050,2780,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,19.962733,3,3,ovary,primary tumour,HGSOC,GSE213243_HGSOC_tumour
GSE213243_tumour_AAATGGACACACGCCA-1,GSE213243,5854,2467,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,4.936795,3,3,ovary,primary tumour,HGSOC,GSE213243_HGSOC_tumour
GSE213243_tumour_AACAAAGCAATTTCCT-1,GSE213243,6073,2541,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,6.323069,3,3,ovary,primary tumour,HGSOC,GSE213243_HGSOC_tumour
GSE213243_tumour_AACACACGTAGCTTTG-1,GSE213243,13497,3862,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,5.319701,3,3,ovary,primary tumour,HGSOC,GSE213243_HGSOC_tumour
GSE213243_tumour_AACACACTCGCTGTTC-1,GSE213243,8644,3306,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,10.596946,3,3,ovary,primary tumour,HGSOC,GSE213243_HGSOC_tumour
GSE213243_tumour_AACAGGGCAACCCTAA-1,GSE213243,6263,2562,tumour,HGSOC,pt-1,GSE213243_HGSOC_tumour,3.544627,3,3,ovary,primary tumour,HGSOC,GSE213243_HGSOC_tumour


,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.5,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE213243_ascites_TTTGATCGTTAGGCCC-2,GSE213243,20342,4702,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,5.899125,5,5,ascites fluid,ascites,HGSOC,GSE213243_HGSOC_ascites
GSE213243_ascites_TTTGATCTCTCGGCTT-2,GSE213243,1614,820,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,34.262701,1,1,ascites fluid,ascites,HGSOC,GSE213243_HGSOC_ascites
GSE213243_ascites_TTTGGAGCACGTCTCT-2,GSE213243,10549,3639,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,11.119537,6,6,ascites fluid,ascites,HGSOC,GSE213243_HGSOC_ascites
GSE213243_ascites_TTTGGAGGTCCTGGGT-2,GSE213243,4613,2061,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,12.421418,1,1,ascites fluid,ascites,HGSOC,GSE213243_HGSOC_ascites
GSE213243_ascites_TTTGGTTCATCCTATT-2,GSE213243,6073,2678,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,11.954553,1,1,ascites fluid,ascites,HGSOC,GSE213243_HGSOC_ascites
GSE213243_ascites_TTTGTTGCATGATGCT-2,GSE213243,14293,4430,ascites,HGSOC,pt-1,GSE213243_HGSOC_ascites,5.044427,6,6,ascites fluid,ascites,HGSOC,GSE213243_HGSOC_ascites


In [61]:
#exclude any samples with <100 cells
table(HGSOC$integration_id)
#none to exclude


GSE213243_HGSOC_ascites  GSE213243_HGSOC_tumour 
                   2688                     804 

In [62]:
#join layers and then split them by integration_id
Layers(HGSOC[["RNA"]])
#join layers
HGSOC[["RNA"]] <- JoinLayers(HGSOC[["RNA"]])
Layers(HGSOC[["RNA"]])
#split layers
HGSOC[["RNA"]] <- split(HGSOC[["RNA"]], f = HGSOC$integration_id)
Layers(HGSOC[["RNA"]])


[1] "counts.1"     "counts.2"     "data.1"       "scale.data.1" "data.2"      
[6] "scale.data.2"

[1] "scale.data" "data"       "counts"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "counts.GSE213243_HGSOC_tumour"  "counts.GSE213243_HGSOC_ascites"
[3] "scale.data"                     "data.GSE213243_HGSOC_tumour"   
[5] "data.GSE213243_HGSOC_ascites"

In [63]:
#record number of cells
table(HGSOC$integration_id)


GSE213243_HGSOC_ascites  GSE213243_HGSOC_tumour 
                   2688                     804 

In [64]:
#re-export seurat object ready for integration
saveRDS(HGSOC, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/GSE213243_myeloid_int.RDS")

In [65]:
#remove all objects in R
rm(list = ls())

## GSE217517

In [66]:
HGSOC <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE217517_myeloid.RDS")

In [67]:
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)

An object of class Seurat 
36601 features across 8457 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)
 17 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, counts.7, counts.8, data.1, data.2, data.3, data.4, data.5, data.6, data.7, data.8, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE217517"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.5,seurat_clusters,RNA_snn_res.0.2
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>
GSE217517_pt1_AAACGAAAGAACCCGA-1,GSE217517,7268,2217,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,3.769950,9,1,1
GSE217517_pt1_AAAGAACCAGGGCTTC-1,GSE217517,20132,4339,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,7.634612,5,1,1
GSE217517_pt1_AAAGAACTCCATGAGT-1,GSE217517,4183,1410,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,35.142242,9,1,1
GSE217517_pt1_AAAGGATTCTATTTCG-1,GSE217517,3037,1274,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,6.914718,9,1,1
GSE217517_pt1_AAATGGACACTGAGGA-1,GSE217517,9516,2822,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,2.847835,5,1,1
GSE217517_pt1_AACAGGGGTCATCGGC-1,GSE217517,22104,4611,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,7.695440,9,1,1


In [68]:
table(HGSOC$sample_type)
table(HGSOC$cancer_type)
table(HGSOC$patient_id)
table(HGSOC$sample_id)


tumour 
  8457 


HGSOC 
 8457 


 pt1  pt2  pt3  pt4  pt5  pt6  pt7  pt8 
 842  966 2678 1517 1004   37 1054  359 


GSE217517_HGSOC_pt1 GSE217517_HGSOC_pt2 GSE217517_HGSOC_pt3 GSE217517_HGSOC_pt4 
                842                 966                2678                1517 
GSE217517_HGSOC_pt5 GSE217517_HGSOC_pt6 GSE217517_HGSOC_pt7 GSE217517_HGSOC_pt8 
               1004                  37                1054                 359 

In [70]:
#set site metadata
HGSOC@meta.data$site <- "ovary"
HGSOC@meta.data$sample_type_major <- "primary tumour"

In [71]:
#set subtype metadata
HGSOC@meta.data$cancer_subtype <- "HGSOC"

In [72]:
#set integration_id metadata
HGSOC@meta.data$integration_id <- HGSOC@meta.data$sample_id

In [73]:
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)

An object of class Seurat 
36601 features across 8457 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)
 17 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, counts.7, counts.8, data.1, data.2, data.3, data.4, data.5, data.6, data.7, data.8, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE217517"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.5,seurat_clusters,RNA_snn_res.0.2,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
GSE217517_pt1_AAACGAAAGAACCCGA-1,GSE217517,7268,2217,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,3.769950,9,1,1,ovary,primary tumour,HGSOC,GSE217517_HGSOC_pt1
GSE217517_pt1_AAAGAACCAGGGCTTC-1,GSE217517,20132,4339,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,7.634612,5,1,1,ovary,primary tumour,HGSOC,GSE217517_HGSOC_pt1
GSE217517_pt1_AAAGAACTCCATGAGT-1,GSE217517,4183,1410,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,35.142242,9,1,1,ovary,primary tumour,HGSOC,GSE217517_HGSOC_pt1
GSE217517_pt1_AAAGGATTCTATTTCG-1,GSE217517,3037,1274,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,6.914718,9,1,1,ovary,primary tumour,HGSOC,GSE217517_HGSOC_pt1
GSE217517_pt1_AAATGGACACTGAGGA-1,GSE217517,9516,2822,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,2.847835,5,1,1,ovary,primary tumour,HGSOC,GSE217517_HGSOC_pt1
GSE217517_pt1_AACAGGGGTCATCGGC-1,GSE217517,22104,4611,tumour,HGSOC,pt1,GSE217517_HGSOC_pt1,7.695440,9,1,1,ovary,primary tumour,HGSOC,GSE217517_HGSOC_pt1


In [74]:
#exclude any samples with <100 cells
table(HGSOC$integration_id)
#exclude patient 6
HGSOC <- subset(HGSOC, !(subset = integration_id %in% c("GSE217517_HGSOC_pt6")))
table(HGSOC$integration_id)


GSE217517_HGSOC_pt1 GSE217517_HGSOC_pt2 GSE217517_HGSOC_pt3 GSE217517_HGSOC_pt4 
                842                 966                2678                1517 
GSE217517_HGSOC_pt5 GSE217517_HGSOC_pt6 GSE217517_HGSOC_pt7 GSE217517_HGSOC_pt8 
               1004                  37                1054                 359 


GSE217517_HGSOC_pt1 GSE217517_HGSOC_pt2 GSE217517_HGSOC_pt3 GSE217517_HGSOC_pt4 
                842                 966                2678                1517 
GSE217517_HGSOC_pt5 GSE217517_HGSOC_pt7 GSE217517_HGSOC_pt8 
               1004                1054                 359 

In [75]:
#join layers and then split them by integration_id
Layers(HGSOC[["RNA"]])
#join layers
HGSOC[["RNA"]] <- JoinLayers(HGSOC[["RNA"]])
Layers(HGSOC[["RNA"]])
#split layers
HGSOC[["RNA"]] <- split(HGSOC[["RNA"]], f = HGSOC$integration_id)
Layers(HGSOC[["RNA"]])


[1] "counts.1"   "counts.2"   "counts.3"   "counts.4"   "counts.5"  
 [6] "counts.7"   "counts.8"   "data.1"     "data.2"     "data.3"    
[11] "data.4"     "data.5"     "data.7"     "data.8"     "scale.data"

[1] "data"       "counts"     "scale.data"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "data.GSE217517_HGSOC_pt1"   "data.GSE217517_HGSOC_pt2"  
 [3] "data.GSE217517_HGSOC_pt3"   "data.GSE217517_HGSOC_pt4"  
 [5] "data.GSE217517_HGSOC_pt5"   "data.GSE217517_HGSOC_pt7"  
 [7] "data.GSE217517_HGSOC_pt8"   "scale.data"                
 [9] "counts.GSE217517_HGSOC_pt1" "counts.GSE217517_HGSOC_pt2"
[11] "counts.GSE217517_HGSOC_pt3" "counts.GSE217517_HGSOC_pt4"
[13] "counts.GSE217517_HGSOC_pt5" "counts.GSE217517_HGSOC_pt7"
[15] "counts.GSE217517_HGSOC_pt8"

In [76]:
#record number of cells
table(HGSOC$integration_id)


GSE217517_HGSOC_pt1 GSE217517_HGSOC_pt2 GSE217517_HGSOC_pt3 GSE217517_HGSOC_pt4 
                842                 966                2678                1517 
GSE217517_HGSOC_pt5 GSE217517_HGSOC_pt7 GSE217517_HGSOC_pt8 
               1004                1054                 359 

In [77]:
#re-export seurat object ready for integration
saveRDS(HGSOC, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/GSE217517_myeloid_int.RDS")

In [78]:
#remove all objects in R
rm(list = ls())

## PRJCA005422

In [79]:
HGSOC_As <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/PRJCA005422_ascites_myeloid.RDS")
HGSOC_Tu <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/PRJCA005422_tumour_myeloid.RDS")

In [80]:
HGSOC_As
HGSOC_As@project.name
head(HGSOC_As@meta.data)

HGSOC_Tu
HGSOC_Tu@project.name
head(HGSOC_Tu@meta.data)

An object of class Seurat 
27127 features across 16120 samples within 1 assay 
Active assay: RNA (27127 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "SeuratProject"

,orig.ident,nCount_RNA,nFeature_RNA,Cellname,Samples,Groups,Patients,percent.mt,percent.ribo,percent.HSP,⋯,maintypes_2,maintypes_3,UMAP_1,UMAP_2,sample_type,cancer_type,patient_id,sample_id,RNA_snn_res.0.2,seurat_clusters
,<fct>,<dbl>,<int>,<chr>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,⋯,<fct>,<chr>,<dbl>,<dbl>,<fct>,<chr>,<fct>,<chr>,<fct>,<fct>
PRJCA005422_EOC1_FS_cell_AACACGTGTCGGCACT,EOC1,24353,1770,EOC1_FS_cell_AACACGTGTCGGCACT,HGSOC1_AS,Ascites,HGSOC1,0.8007227,3.859894,0.1519320,⋯,B,Lymphoid cells,0.1662883,13.255426,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8
PRJCA005422_EOC1_FS_cell_AACCGCGTCCCTAACC,EOC1,531,365,EOC1_FS_cell_AACCGCGTCCCTAACC,HGSOC1_AS,Ascites,HGSOC1,5.6497175,18.796992,0.1879699,⋯,B,Lymphoid cells,-6.0097427,11.557367,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,4,4
PRJCA005422_EOC1_FS_cell_AACTCCCAGTTTCCTT,EOC1,9273,3128,EOC1_FS_cell_AACTCCCAGTTTCCTT,HGSOC1_AS,Ascites,HGSOC1,3.5479349,10.673854,0.4743935,⋯,Proliferative cells,Proliferative cells,2.3708313,2.942190,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8
PRJCA005422_EOC1_FS_cell_AAGGCAGGTTAAAGTG,EOC1,4757,2120,EOC1_FS_cell_AAGGCAGGTTAAAGTG,HGSOC1_AS,Ascites,HGSOC1,9.7750683,15.762926,0.6094998,⋯,Proliferative cells,Proliferative cells,2.5225659,2.890291,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8
PRJCA005422_EOC1_FS_cell_AAGGCAGTCAACACTG,EOC1,19574,3727,EOC1_FS_cell_AAGGCAGTCAACACTG,HGSOC1_AS,Ascites,HGSOC1,7.1319097,24.638570,0.3473819,⋯,Proliferative cells,Proliferative cells,1.6956519,3.476046,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8
PRJCA005422_EOC1_FS_cell_ACACCAAAGCTAACTC,EOC1,22514,3971,EOC1_FS_cell_ACACCAAAGCTAACTC,HGSOC1_AS,Ascites,HGSOC1,5.7208848,20.034642,0.7683425,⋯,Proliferative cells,Proliferative cells,2.0638140,12.254530,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8


An object of class Seurat 
27127 features across 13256 samples within 1 assay 
Active assay: RNA (27127 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "SeuratProject"

,orig.ident,nCount_RNA,nFeature_RNA,Cellname,Samples,Groups,Patients,percent.mt,percent.ribo,percent.HSP,⋯,maintypes_2,maintypes_3,UMAP_1,UMAP_2,sample_type,cancer_type,patient_id,sample_id,RNA_snn_res.0.2,seurat_clusters
,<fct>,<dbl>,<int>,<chr>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,⋯,<fct>,<chr>,<dbl>,<dbl>,<fct>,<chr>,<fct>,<chr>,<fct>,<fct>
PRJCA005422_EOC1_OC_cell_CCACGGACACCAGGCT,EOC1,631,422,EOC1_OC_cell_CCACGGACACCAGGCT,HGSOC1_PT,Primary Tumor,HGSOC1,5.0713154,16.45570,1.2658228,⋯,Proliferative cells,Proliferative cells,1.570391,2.864907,Primary Tumor,HGSOC,HGSOC1,PRJCA005422_HGSOC1_PT,0,0
PRJCA005422_EOC1_OC_cell_CCTACACAGAGTCTGG,EOC1,639,368,EOC1_OC_cell_CCTACACAGAGTCTGG,HGSOC1_PT,Primary Tumor,HGSOC1,3.7558685,25.00000,1.4062500,⋯,Proliferative cells,Proliferative cells,2.063956,3.946421,Primary Tumor,HGSOC,HGSOC1,PRJCA005422_HGSOC1_PT,0,0
PRJCA005422_EOC1_TM_cell_AGTTGGTTCACGCATA,EOC1,651,394,EOC1_TM_cell_AGTTGGTTCACGCATA,HGSOC1_MT,Metastatic Tumor,HGSOC1,0.4608295,26.72811,0.1536098,⋯,Proliferative cells,Proliferative cells,1.906724,3.602650,Metastatic Tumor,HGSOC,HGSOC1,PRJCA005422_HGSOC1_MT,0,0
PRJCA005422_EOC1_TM_cell_CGATCGGCACGCTTTC,EOC1,1480,787,EOC1_TM_cell_CGATCGGCACGCTTTC,HGSOC1_MT,Metastatic Tumor,HGSOC1,0.4054054,25.60811,0.6081081,⋯,Proliferative cells,Proliferative cells,2.014700,3.495903,Metastatic Tumor,HGSOC,HGSOC1,PRJCA005422_HGSOC1_MT,0,0
PRJCA005422_EOC1_TM_cell_CTCTAATTCTTTACGT,EOC1,1067,522,EOC1_TM_cell_CTCTAATTCTTTACGT,HGSOC1_MT,Metastatic Tumor,HGSOC1,1.2183693,38.23805,0.1874414,⋯,Proliferative cells,Proliferative cells,1.414386,3.496294,Metastatic Tumor,HGSOC,HGSOC1,PRJCA005422_HGSOC1_MT,0,0
PRJCA005422_EOC1_TM_cell_GCCTCTACACGGTTTA,EOC1,1629,792,EOC1_TM_cell_GCCTCTACACGGTTTA,HGSOC1_MT,Metastatic Tumor,HGSOC1,0.0000000,26.27379,0.9821977,⋯,Proliferative cells,Proliferative cells,1.583880,3.464499,Metastatic Tumor,HGSOC,HGSOC1,PRJCA005422_HGSOC1_MT,0,0


In [81]:
table(HGSOC_As$sample_type)
table(HGSOC_As$cancer_type)
table(HGSOC_As$patient_id)
table(HGSOC_As$sample_id)

table(HGSOC_Tu$sample_type)
table(HGSOC_Tu$cancer_type)
table(HGSOC_Tu$patient_id)
table(HGSOC_Tu$sample_id)


   Primary Tumor Metastatic Tumor       Lymph Node          Ascites 
               0                0                0            16120 
            PBMC 
               0 


HGSOC 
16120 


 HGSOC1  HGSOC2  HGSOC3  HGSOC4  HGSOC5  HGSOC6  HGSOC7  HGSOC8  HGSOC9 HGSOC10 
   1149    6695     662       0    1743     829       0    1110    3589     343 
   ECO1    UOC1   OCCC1      C1 
      0       0       0       0 


 PRJCA005422_HGSOC1_AS PRJCA005422_HGSOC10_AS  PRJCA005422_HGSOC2_AS 
                  1149                    343                   6695 
 PRJCA005422_HGSOC3_AS  PRJCA005422_HGSOC5_AS  PRJCA005422_HGSOC6_AS 
                   662                   1743                    829 
 PRJCA005422_HGSOC8_AS  PRJCA005422_HGSOC9_AS 
                  1110                   3589 


   Primary Tumor Metastatic Tumor       Lymph Node          Ascites 
            8041             5215                0                0 
            PBMC 
               0 


HGSOC 
13256 


 HGSOC1  HGSOC2  HGSOC3  HGSOC4  HGSOC5  HGSOC6  HGSOC7  HGSOC8  HGSOC9 HGSOC10 
   2639     633    3523    1104      70    2150    1179     121    1087     750 
   ECO1    UOC1   OCCC1      C1 
      0       0       0       0 


 PRJCA005422_HGSOC1_MT  PRJCA005422_HGSOC1_PT PRJCA005422_HGSOC10_PT 
                  1231                   1408                    750 
 PRJCA005422_HGSOC2_PT  PRJCA005422_HGSOC3_MT  PRJCA005422_HGSOC3_PT 
                   633                   1711                   1812 
 PRJCA005422_HGSOC4_MT  PRJCA005422_HGSOC4_PT  PRJCA005422_HGSOC5_PT 
                   816                    288                     70 
 PRJCA005422_HGSOC6_MT  PRJCA005422_HGSOC6_PT  PRJCA005422_HGSOC7_PT 
                  1457                    693                   1179 
 PRJCA005422_HGSOC8_PT  PRJCA005422_HGSOC9_PT 
                   121                   1087 

In [82]:
#set site metadata
HGSOC_As@meta.data$site <- "ascites fluid"
HGSOC_As@meta.data$sample_type_major <- "ascites"

#HGSOC_Tu need to split up primary and mets by location
HGSOC_Pr <- subset(HGSOC_Tu, subset = sample_type %in% c("Primary Tumor"))
HGSOC_Me <- subset(HGSOC_Tu, subset = sample_type %in% c("Metastatic Tumor"))

HGSOC_Pr@meta.data$site <- "ovary"
HGSOC_Me@meta.data$site <- "omentum"

HGSOC_Pr@meta.data$sample_type_major <- "primary tumour"
HGSOC_Me@meta.data$sample_type_major <- "metastatic tumour"

#Merge seurat objects back together
HGSOC <- merge(HGSOC_As, y = c(HGSOC_Pr, HGSOC_Me), project = "PRJCA005422")

In [84]:
#set subtype metadata
HGSOC@meta.data$cancer_subtype <- "HGSOC"

In [85]:
#set integration_id metadata
HGSOC@meta.data$integration_id <- HGSOC@meta.data$sample_id

In [86]:
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)

An object of class Seurat 
27127 features across 29376 samples within 1 assay 
Active assay: RNA (27127 features, 2000 variable features)
 9 layers present: counts.1, counts.2, counts.3, data.1, scale.data.1, data.2, scale.data.2, data.3, scale.data.3

[1] "PRJCA005422"

,orig.ident,nCount_RNA,nFeature_RNA,Cellname,Samples,Groups,Patients,percent.mt,percent.ribo,percent.HSP,⋯,sample_type,cancer_type,patient_id,sample_id,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PRJCA005422_EOC1_FS_cell_AACACGTGTCGGCACT,EOC1,24353,1770,EOC1_FS_cell_AACACGTGTCGGCACT,HGSOC1_AS,Ascites,HGSOC1,0.8007227,3.859894,0.1519320,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AACCGCGTCCCTAACC,EOC1,531,365,EOC1_FS_cell_AACCGCGTCCCTAACC,HGSOC1_AS,Ascites,HGSOC1,5.6497175,18.796992,0.1879699,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,4,4,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AACTCCCAGTTTCCTT,EOC1,9273,3128,EOC1_FS_cell_AACTCCCAGTTTCCTT,HGSOC1_AS,Ascites,HGSOC1,3.5479349,10.673854,0.4743935,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AAGGCAGGTTAAAGTG,EOC1,4757,2120,EOC1_FS_cell_AAGGCAGGTTAAAGTG,HGSOC1_AS,Ascites,HGSOC1,9.7750683,15.762926,0.6094998,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AAGGCAGTCAACACTG,EOC1,19574,3727,EOC1_FS_cell_AAGGCAGTCAACACTG,HGSOC1_AS,Ascites,HGSOC1,7.1319097,24.638570,0.3473819,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_ACACCAAAGCTAACTC,EOC1,22514,3971,EOC1_FS_cell_ACACCAAAGCTAACTC,HGSOC1_AS,Ascites,HGSOC1,5.7208848,20.034642,0.7683425,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS


In [87]:
#exclude any samples with <100 cells
table(HGSOC$integration_id)
#exclude patient HGSOC5 primary tumour
HGSOC <- subset(HGSOC, !(subset = integration_id %in% c("PRJCA005422_HGSOC5_PT")))
table(HGSOC$integration_id)


 PRJCA005422_HGSOC1_AS  PRJCA005422_HGSOC1_MT  PRJCA005422_HGSOC1_PT 
                  1149                   1231                   1408 
PRJCA005422_HGSOC10_AS PRJCA005422_HGSOC10_PT  PRJCA005422_HGSOC2_AS 
                   343                    750                   6695 
 PRJCA005422_HGSOC2_PT  PRJCA005422_HGSOC3_AS  PRJCA005422_HGSOC3_MT 
                   633                    662                   1711 
 PRJCA005422_HGSOC3_PT  PRJCA005422_HGSOC4_MT  PRJCA005422_HGSOC4_PT 
                  1812                    816                    288 
 PRJCA005422_HGSOC5_AS  PRJCA005422_HGSOC5_PT  PRJCA005422_HGSOC6_AS 
                  1743                     70                    829 
 PRJCA005422_HGSOC6_MT  PRJCA005422_HGSOC6_PT  PRJCA005422_HGSOC7_PT 
                  1457                    693                   1179 
 PRJCA005422_HGSOC8_AS  PRJCA005422_HGSOC8_PT  PRJCA005422_HGSOC9_AS 
                  1110                    121                   3589 
 PRJCA005422_HGSOC9


 PRJCA005422_HGSOC1_AS  PRJCA005422_HGSOC1_MT  PRJCA005422_HGSOC1_PT 
                  1149                   1231                   1408 
PRJCA005422_HGSOC10_AS PRJCA005422_HGSOC10_PT  PRJCA005422_HGSOC2_AS 
                   343                    750                   6695 
 PRJCA005422_HGSOC2_PT  PRJCA005422_HGSOC3_AS  PRJCA005422_HGSOC3_MT 
                   633                    662                   1711 
 PRJCA005422_HGSOC3_PT  PRJCA005422_HGSOC4_MT  PRJCA005422_HGSOC4_PT 
                  1812                    816                    288 
 PRJCA005422_HGSOC5_AS  PRJCA005422_HGSOC6_AS  PRJCA005422_HGSOC6_MT 
                  1743                    829                   1457 
 PRJCA005422_HGSOC6_PT  PRJCA005422_HGSOC7_PT  PRJCA005422_HGSOC8_AS 
                   693                   1179                   1110 
 PRJCA005422_HGSOC8_PT  PRJCA005422_HGSOC9_AS  PRJCA005422_HGSOC9_PT 
                   121                   3589                   1087 

In [88]:
#join layers and then split them by integration_id
Layers(HGSOC[["RNA"]])
#join layers
HGSOC[["RNA"]] <- JoinLayers(HGSOC[["RNA"]])
Layers(HGSOC[["RNA"]])
#split layers
HGSOC[["RNA"]] <- split(HGSOC[["RNA"]], f = HGSOC$integration_id)
Layers(HGSOC[["RNA"]])


[1] "counts.1"     "counts.2"     "counts.3"     "data.1"       "scale.data.1"
[6] "data.2"       "scale.data.2" "data.3"       "scale.data.3"

[1] "scale.data" "data"       "counts"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "counts.PRJCA005422_HGSOC1_AS"  "counts.PRJCA005422_HGSOC3_AS" 
 [3] "counts.PRJCA005422_HGSOC2_AS"  "counts.PRJCA005422_HGSOC6_AS" 
 [5] "counts.PRJCA005422_HGSOC5_AS"  "counts.PRJCA005422_HGSOC8_AS" 
 [7] "counts.PRJCA005422_HGSOC9_AS"  "counts.PRJCA005422_HGSOC10_AS"
 [9] "counts.PRJCA005422_HGSOC1_PT"  "counts.PRJCA005422_HGSOC3_PT" 
[11] "counts.PRJCA005422_HGSOC2_PT"  "counts.PRJCA005422_HGSOC7_PT" 
[13] "counts.PRJCA005422_HGSOC6_PT"  "counts.PRJCA005422_HGSOC4_PT" 
[15] "counts.PRJCA005422_HGSOC8_PT"  "counts.PRJCA005422_HGSOC9_PT" 
[17] "counts.PRJCA005422_HGSOC10_PT" "counts.PRJCA005422_HGSOC1_MT" 
[19] "counts.PRJCA005422_HGSOC3_MT"  "counts.PRJCA005422_HGSOC6_MT" 
[21] "counts.PRJCA005422_HGSOC4_MT"  "scale.data"                   
[23] "data.PRJCA005422_HGSOC1_AS"    "data.PRJCA005422_HGSOC3_AS"   
[25] "data.PRJCA005422_HGSOC2_AS"    "data.PRJCA005422_HGSOC6_AS"   
[27] "data.PRJCA005422_HGSOC5_AS"    "data.PRJCA005422_HGSOC8_AS"   
[29] "data.PRJCA005422_HGSOC9_AS"    "data.PRJCA005422_HGSOC10_AS"  
[31] "data.PRJCA005422_HGSOC1_PT"    "data.PRJCA005422_HGSOC3_PT"   
[33] "data.PRJCA005422_HGSOC2_PT"    "data.PRJCA005422_HGSOC7_PT"   
[35] "data.PRJCA005422_HGSOC6_PT"    "data.PRJCA005422_HGSOC4_PT"   
[37] "data.PRJCA005422_HGSOC8_PT"    "data.PRJCA005422_HGSOC9_PT"   
[39] "data.PRJCA005422_HGSOC10_PT"   "data.PRJCA005422_HGSOC1_MT"   
[41] "data.PRJCA005422_HGSOC3_MT"    "data.PRJCA005422_HGSOC6_MT"   
[43] "data.PRJCA005422_HGSOC4_MT"

In [89]:
#record number of cells
HGSOC
HGSOC@project.name
head(HGSOC@meta.data)
tail(HGSOC@meta.data)
table(HGSOC$integration_id)

An object of class Seurat 
27127 features across 29306 samples within 1 assay 
Active assay: RNA (27127 features, 2000 variable features)
 43 layers present: counts.PRJCA005422_HGSOC1_AS, counts.PRJCA005422_HGSOC3_AS, counts.PRJCA005422_HGSOC2_AS, counts.PRJCA005422_HGSOC6_AS, counts.PRJCA005422_HGSOC5_AS, counts.PRJCA005422_HGSOC8_AS, counts.PRJCA005422_HGSOC9_AS, counts.PRJCA005422_HGSOC10_AS, counts.PRJCA005422_HGSOC1_PT, counts.PRJCA005422_HGSOC3_PT, counts.PRJCA005422_HGSOC2_PT, counts.PRJCA005422_HGSOC7_PT, counts.PRJCA005422_HGSOC6_PT, counts.PRJCA005422_HGSOC4_PT, counts.PRJCA005422_HGSOC8_PT, counts.PRJCA005422_HGSOC9_PT, counts.PRJCA005422_HGSOC10_PT, counts.PRJCA005422_HGSOC1_MT, counts.PRJCA005422_HGSOC3_MT, counts.PRJCA005422_HGSOC6_MT, counts.PRJCA005422_HGSOC4_MT, scale.data, data.PRJCA005422_HGSOC1_AS, data.PRJCA005422_HGSOC3_AS, data.PRJCA005422_HGSOC2_AS, data.PRJCA005422_HGSOC6_AS, data.PRJCA005422_HGSOC5_AS, data.PRJCA005422_HGSOC8_AS, data.PRJCA005422_HGSOC9_AS, da

[1] "PRJCA005422"

,orig.ident,nCount_RNA,nFeature_RNA,Cellname,Samples,Groups,Patients,percent.mt,percent.ribo,percent.HSP,⋯,sample_type,cancer_type,patient_id,sample_id,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PRJCA005422_EOC1_FS_cell_AACACGTGTCGGCACT,EOC1,24353,1770,EOC1_FS_cell_AACACGTGTCGGCACT,HGSOC1_AS,Ascites,HGSOC1,0.8007227,3.859894,0.1519320,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AACCGCGTCCCTAACC,EOC1,531,365,EOC1_FS_cell_AACCGCGTCCCTAACC,HGSOC1_AS,Ascites,HGSOC1,5.6497175,18.796992,0.1879699,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,4,4,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AACTCCCAGTTTCCTT,EOC1,9273,3128,EOC1_FS_cell_AACTCCCAGTTTCCTT,HGSOC1_AS,Ascites,HGSOC1,3.5479349,10.673854,0.4743935,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AAGGCAGGTTAAAGTG,EOC1,4757,2120,EOC1_FS_cell_AAGGCAGGTTAAAGTG,HGSOC1_AS,Ascites,HGSOC1,9.7750683,15.762926,0.6094998,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_AAGGCAGTCAACACTG,EOC1,19574,3727,EOC1_FS_cell_AAGGCAGTCAACACTG,HGSOC1_AS,Ascites,HGSOC1,7.1319097,24.638570,0.3473819,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS
PRJCA005422_EOC1_FS_cell_ACACCAAAGCTAACTC,EOC1,22514,3971,EOC1_FS_cell_ACACCAAAGCTAACTC,HGSOC1_AS,Ascites,HGSOC1,5.7208848,20.034642,0.7683425,⋯,Ascites,HGSOC,HGSOC1,PRJCA005422_HGSOC1_AS,8,8,ascites fluid,ascites,HGSOC,PRJCA005422_HGSOC1_AS


,orig.ident,nCount_RNA,nFeature_RNA,Cellname,Samples,Groups,Patients,percent.mt,percent.ribo,percent.HSP,⋯,sample_type,cancer_type,patient_id,sample_id,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PRJCA005422_EOC4_TM_cell_TTTCCTCGTTCAACCA,EOC4,12786,3090,EOC4_TM_cell_TTTCCTCGTTCAACCA,HGSOC4_MT,Metastatic Tumor,HGSOC4,0.9619897,7.382498,0.4223039,⋯,Metastatic Tumor,HGSOC,HGSOC4,PRJCA005422_HGSOC4_MT,0,0,omentum,metastatic tumour,HGSOC,PRJCA005422_HGSOC4_MT
PRJCA005422_EOC4_TM_cell_TTTCCTCTCTGGTATG,EOC4,13288,3440,EOC4_TM_cell_TTTCCTCTCTGGTATG,HGSOC4_MT,Metastatic Tumor,HGSOC4,3.6950632,15.937994,0.3988261,⋯,Metastatic Tumor,HGSOC,HGSOC4,PRJCA005422_HGSOC4_MT,0,0,omentum,metastatic tumour,HGSOC,PRJCA005422_HGSOC4_MT
PRJCA005422_EOC4_TM_cell_TTTCCTCTCTGGTTCC,EOC4,16083,3128,EOC4_TM_cell_TTTCCTCTCTGGTTCC,HGSOC4_MT,Metastatic Tumor,HGSOC4,2.8352919,7.181049,0.3295200,⋯,Metastatic Tumor,HGSOC,HGSOC4,PRJCA005422_HGSOC4_MT,0,0,omentum,metastatic tumour,HGSOC,PRJCA005422_HGSOC4_MT
PRJCA005422_EOC4_TM_cell_TTTCCTCTCTGTCTCG,EOC4,23953,4481,EOC4_TM_cell_TTTCCTCTCTGTCTCG,HGSOC4_MT,Metastatic Tumor,HGSOC4,2.9140400,8.449176,0.4049259,⋯,Metastatic Tumor,HGSOC,HGSOC4,PRJCA005422_HGSOC4_MT,0,0,omentum,metastatic tumour,HGSOC,PRJCA005422_HGSOC4_MT
PRJCA005422_EOC4_TM_cell_TTTGCGCAGGAGTCTG,EOC4,17788,3363,EOC4_TM_cell_TTTGCGCAGGAGTCTG,HGSOC4_MT,Metastatic Tumor,HGSOC4,4.1207556,15.797167,0.3879020,⋯,Metastatic Tumor,HGSOC,HGSOC4,PRJCA005422_HGSOC4_MT,0,0,omentum,metastatic tumour,HGSOC,PRJCA005422_HGSOC4_MT
PRJCA005422_EOC4_TM_cell_TTTGTCATCCCAACGG,EOC4,16918,3537,EOC4_TM_cell_TTTGTCATCCCAACGG,HGSOC4_MT,Metastatic Tumor,HGSOC4,7.4063128,20.745907,0.4787517,⋯,Metastatic Tumor,HGSOC,HGSOC4,PRJCA005422_HGSOC4_MT,0,0,omentum,metastatic tumour,HGSOC,PRJCA005422_HGSOC4_MT



 PRJCA005422_HGSOC1_AS  PRJCA005422_HGSOC1_MT  PRJCA005422_HGSOC1_PT 
                  1149                   1231                   1408 
PRJCA005422_HGSOC10_AS PRJCA005422_HGSOC10_PT  PRJCA005422_HGSOC2_AS 
                   343                    750                   6695 
 PRJCA005422_HGSOC2_PT  PRJCA005422_HGSOC3_AS  PRJCA005422_HGSOC3_MT 
                   633                    662                   1711 
 PRJCA005422_HGSOC3_PT  PRJCA005422_HGSOC4_MT  PRJCA005422_HGSOC4_PT 
                  1812                    816                    288 
 PRJCA005422_HGSOC5_AS  PRJCA005422_HGSOC6_AS  PRJCA005422_HGSOC6_MT 
                  1743                    829                   1457 
 PRJCA005422_HGSOC6_PT  PRJCA005422_HGSOC7_PT  PRJCA005422_HGSOC8_AS 
                   693                   1179                   1110 
 PRJCA005422_HGSOC8_PT  PRJCA005422_HGSOC9_AS  PRJCA005422_HGSOC9_PT 
                   121                   3589                   1087 

In [90]:
#re-export seurat object ready for integration
saveRDS(HGSOC, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/PRJCA005422_myeloid_int.RDS")

In [91]:
#remove all objects in R
rm(list = ls())

## GSE200218

In [123]:
MEL <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE200218_myeloid.RDS")

In [124]:
MEL
MEL@project.name
head(MEL@meta.data)

An object of class Seurat 
36601 features across 10371 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)
 11 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, data.1, data.2, data.3, data.4, data.5, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE200218"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GSE200218_MBM01_AAACCTGAGCTGCAAG-1,GSE200218,15791,4063,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,4.439238,0,0
GSE200218_MBM01_AAACCTGCAATCGGTT-1,GSE200218,29993,5932,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.270763,2,2
GSE200218_MBM01_AAACCTGGTACTTGAC-1,GSE200218,21267,5177,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.282080,7,7
GSE200218_MBM01_AAACCTGGTTAAGATG-1,GSE200218,25744,5563,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.899938,0,0
GSE200218_MBM01_AAACCTGTCACGGTTA-1,GSE200218,14369,3779,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.257012,0,0
GSE200218_MBM01_AAACCTGTCCGCATCT-1,GSE200218,3921,2039,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,4.412140,0,0


In [125]:
table(MEL$sample_type)
table(MEL$cancer_type)
table(MEL$patient_id)
table(MEL$sample_id)


metastasis 
     10371 


melanoma brain mets 
              10371 


MBM01 MBM02 MBM03 MBM04 MBM05 
 1411  2035  1945  3143  1837 


GSE200218_MBM01 GSE200218_MBM02 GSE200218_MBM03 GSE200218_MBM04 GSE200218_MBM05 
           1411            2035            1945            3143            1837 

In [126]:
#set site and sample_type_major metadata
MEL@meta.data$site <- "brain"
MEL@meta.data$sample_type_major <- "metastatic tumour"

In [127]:
#set subtype metadata
MEL@meta.data$cancer_subtype <- "Melanoma"

In [128]:
#set integration_id metadata
MEL@meta.data$integration_id <- MEL@meta.data$sample_id

In [129]:
MEL
MEL@project.name
head(MEL@meta.data)

An object of class Seurat 
36601 features across 10371 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)
 11 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, data.1, data.2, data.3, data.4, data.5, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE200218"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
GSE200218_MBM01_AAACCTGAGCTGCAAG-1,GSE200218,15791,4063,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,4.439238,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGCAATCGGTT-1,GSE200218,29993,5932,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.270763,2,2,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGGTACTTGAC-1,GSE200218,21267,5177,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.282080,7,7,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGGTTAAGATG-1,GSE200218,25744,5563,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.899938,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGTCACGGTTA-1,GSE200218,14369,3779,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.257012,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGTCCGCATCT-1,GSE200218,3921,2039,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,4.412140,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01


In [130]:
#exclude any samples with <100 cells
table(MEL$integration_id)
#none to exclude


GSE200218_MBM01 GSE200218_MBM02 GSE200218_MBM03 GSE200218_MBM04 GSE200218_MBM05 
           1411            2035            1945            3143            1837 

In [131]:
#join layers and then split them by integration_id
Layers(MEL[["RNA"]])
#join layers
MEL[["RNA"]] <- JoinLayers(MEL[["RNA"]])
Layers(MEL[["RNA"]])
#split layers
MEL[["RNA"]] <- split(MEL[["RNA"]], f = MEL$integration_id)
Layers(MEL[["RNA"]])


[1] "counts.1"   "counts.2"   "counts.3"   "counts.4"   "counts.5"  
 [6] "data.1"     "data.2"     "data.3"     "data.4"     "data.5"    
[11] "scale.data"

[1] "data"       "counts"     "scale.data"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "data.GSE200218_MBM01"   "data.GSE200218_MBM02"   "data.GSE200218_MBM03"  
 [4] "data.GSE200218_MBM04"   "data.GSE200218_MBM05"   "scale.data"            
 [7] "counts.GSE200218_MBM01" "counts.GSE200218_MBM02" "counts.GSE200218_MBM03"
[10] "counts.GSE200218_MBM04" "counts.GSE200218_MBM05"

In [132]:
#record number of cells
MEL
MEL@project.name
head(MEL@meta.data)
tail(MEL@meta.data)
table(MEL$integration_id)

An object of class Seurat 
36601 features across 10371 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)
 11 layers present: data.GSE200218_MBM01, data.GSE200218_MBM02, data.GSE200218_MBM03, data.GSE200218_MBM04, data.GSE200218_MBM05, scale.data, counts.GSE200218_MBM01, counts.GSE200218_MBM02, counts.GSE200218_MBM03, counts.GSE200218_MBM04, counts.GSE200218_MBM05
 2 dimensional reductions calculated: pca, umap

[1] "GSE200218"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
GSE200218_MBM01_AAACCTGAGCTGCAAG-1,GSE200218,15791,4063,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,4.439238,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGCAATCGGTT-1,GSE200218,29993,5932,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.270763,2,2,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGGTACTTGAC-1,GSE200218,21267,5177,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.282080,7,7,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGGTTAAGATG-1,GSE200218,25744,5563,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.899938,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGTCACGGTTA-1,GSE200218,14369,3779,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,3.257012,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01
GSE200218_MBM01_AAACCTGTCCGCATCT-1,GSE200218,3921,2039,metastasis,melanoma brain mets,MBM01,GSE200218_MBM01,4.412140,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM01


,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
GSE200218_MBM05_TTTGCGCTCATGCTCC-1,GSE200218,9728,2563,metastasis,melanoma brain mets,MBM05,GSE200218_MBM05,9.477796,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM05
GSE200218_MBM05_TTTGCGCTCCTCGCAT-1,GSE200218,13511,3309,metastasis,melanoma brain mets,MBM05,GSE200218_MBM05,3.449042,2,2,brain,metastatic tumour,Melanoma,GSE200218_MBM05
GSE200218_MBM05_TTTGGTTTCTCTAAGG-1,GSE200218,15440,3540,metastasis,melanoma brain mets,MBM05,GSE200218_MBM05,4.410622,2,2,brain,metastatic tumour,Melanoma,GSE200218_MBM05
GSE200218_MBM05_TTTGTCAAGATGGCGT-1,GSE200218,10913,2889,metastasis,melanoma brain mets,MBM05,GSE200218_MBM05,4.370934,2,2,brain,metastatic tumour,Melanoma,GSE200218_MBM05
GSE200218_MBM05_TTTGTCAAGTTAGGTA-1,GSE200218,5539,2048,metastasis,melanoma brain mets,MBM05,GSE200218_MBM05,5.361979,0,0,brain,metastatic tumour,Melanoma,GSE200218_MBM05
GSE200218_MBM05_TTTGTCATCTTGGGTA-1,GSE200218,1780,1076,metastasis,melanoma brain mets,MBM05,GSE200218_MBM05,10.561798,7,7,brain,metastatic tumour,Melanoma,GSE200218_MBM05



GSE200218_MBM01 GSE200218_MBM02 GSE200218_MBM03 GSE200218_MBM04 GSE200218_MBM05 
           1411            2035            1945            3143            1837 

In [134]:
#re-export seurat object ready for integration
saveRDS(MEL, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/GSE200218_myeloid_int.RDS")

In [135]:
#remove all objects in R
rm(list = ls())

## GSE215120

In [136]:
MEL_Ac <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE215120_AcMEL_myeloid.RDS")
MEL_Cu <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/GSE215120_CuMEL_myeloid.RDS")

In [137]:
MEL_Ac
MEL_Ac@project.name
head(MEL_Ac@meta.data)

MEL_Cu
MEL_Cu@project.name
head(MEL_Cu@meta.data)

An object of class Seurat 
33538 features across 787 samples within 1 assay 
Active assay: RNA (33538 features, 2000 variable features)
 13 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, data.1, data.2, data.3, data.4, data.5, data.6, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE215120"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GSE215120_AM1_AAACCTGGTTGCTCCT-1,GSE215120,20298,3789,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,0.9754656,12,12
GSE215120_AM1_AAAGATGTCCAAATGC-1,GSE215120,5574,1721,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,6.0459275,12,12
GSE215120_AM1_AAAGTAGTCGGTGTTA-1,GSE215120,13432,2759,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.1515783,12,12
GSE215120_AM1_AAATGCCCAGAGCCAA-1,GSE215120,17143,2659,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.2249898,12,12
GSE215120_AM1_AAATGCCGTTTGGCGC-1,GSE215120,3603,1012,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,3.6081044,12,12
GSE215120_AM1_AAATGCCTCATGTCCC-1,GSE215120,14482,2882,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.0357685,12,12


An object of class Seurat 
33538 features across 427 samples within 1 assay 
Active assay: RNA (33538 features, 2000 variable features)
 9 layers present: counts.1, counts.2, counts.3, counts.4, data.1, data.2, data.3, data.4, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "GSE215120"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GSE215120_CM1_AAATGCCCATTACCTT-1,GSE215120,7596,1914,tumour,Cutaneous Melanoma,CM1,GSE215120_Cut_MEL_CM1,3.1200632,11,11
GSE215120_CM1_AACTCCCAGCCGATTT-1,GSE215120,4828,1341,tumour,Cutaneous Melanoma,CM1,GSE215120_Cut_MEL_CM1,0.9113505,11,11
GSE215120_CM1_AACTCCCTCGGCGCAT-1,GSE215120,7064,1684,tumour,Cutaneous Melanoma,CM1,GSE215120_Cut_MEL_CM1,2.5622877,11,11
GSE215120_CM1_AATCCAGTCAGGCCCA-1,GSE215120,10178,2223,tumour,Cutaneous Melanoma,CM1,GSE215120_Cut_MEL_CM1,2.0141482,11,11
GSE215120_CM1_ACACCAAGTCTTCTCG-1,GSE215120,5097,1378,tumour,Cutaneous Melanoma,CM1,GSE215120_Cut_MEL_CM1,0.3923877,11,11
GSE215120_CM1_ACACCCTCAATACGCT-1,GSE215120,7358,1667,tumour,Cutaneous Melanoma,CM1,GSE215120_Cut_MEL_CM1,2.1337320,11,11


In [138]:
table(MEL_Ac$sample_type)
table(MEL_Ac$cancer_type)
table(MEL_Ac$patient_id)
table(MEL_Ac$sample_id)

table(MEL_Cu$sample_type)
table(MEL_Cu$cancer_type)
table(MEL_Cu$patient_id)
table(MEL_Cu$sample_id)


tumour 
   787 


Acral Melanoma 
           787 


AM1 AM2 AM3 AM4 AM5 AM6 
260  23 101   9 279 115 


GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM2 GSE215120_Acral_MEL_AM3 
                    260                      23                     101 
GSE215120_Acral_MEL_AM4 GSE215120_Acral_MEL_AM5 GSE215120_Acral_MEL_AM6 
                      9                     279                     115 


LN metastasis        tumour 
          162           265 


Cutaneous Melanoma 
               427 


CM1 CM2 CM3 
295  32 100 


 GSE215120_Cut_MEL_CM1  GSE215120_Cut_MEL_CM2  GSE215120_Cut_MEL_CM3 
                   133                     32                    100 
GSE215120_MEL_mets_CM1 
                   162 

In [139]:
#split by cancer_type
MEL_Cu_Tu <- subset(MEL_Cu, subset = sample_type %in% c("tumour"))
MEL_Cu_LN <- subset(MEL_Cu, subset = sample_type %in% c("LN metastasis"))

#set site and sample_type_major metadata
MEL_Ac@meta.data$site <- "skin"
MEL_Cu_Tu@meta.data$site <- "skin"
MEL_Cu_LN@meta.data$site <- "lymph node"

MEL_Ac@meta.data$sample_type_major <- "primary tumour"
MEL_Cu_Tu@meta.data$sample_type_major <- "primary tumour"
MEL_Cu_LN@meta.data$sample_type_major <- "metastatic tumour"

#set subtype metadata
MEL_Ac@meta.data$cancer_subtype <- "Acral Melanoma"
MEL_Cu_Tu@meta.data$cancer_subtype <- "Melanoma"
MEL_Cu_LN@meta.data$cancer_subtype <- "Melanoma"

#Merge seurat objects back together
MEL <- merge(MEL_Ac, y = c(MEL_Cu_Tu, MEL_Cu_LN), project = "GSE215120")

In [140]:
#set integration_id metadata
MEL@meta.data$integration_id <- MEL@meta.data$sample_id

In [141]:
MEL
MEL@project.name
head(MEL@meta.data)

An object of class Seurat 
33538 features across 1214 samples within 1 assay 
Active assay: RNA (33538 features, 2000 variable features)
 23 layers present: counts.1.1, counts.1.2, counts.2.1, counts.2.2, counts.3.1, counts.3.2, counts.4.1, counts.4.3, counts.5.1, counts.6.1, data.1.1, data.2.1, data.3.1, data.4.1, data.5.1, data.6.1, scale.data.1, data.1.2, data.2.2, data.3.2, scale.data.2, data.4.3, scale.data.3

[1] "GSE215120"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE215120_AM1_AAACCTGGTTGCTCCT-1,GSE215120,20298,3789,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,0.9754656,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGATGTCCAAATGC-1,GSE215120,5574,1721,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,6.0459275,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGTAGTCGGTGTTA-1,GSE215120,13432,2759,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.1515783,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCCAGAGCCAA-1,GSE215120,17143,2659,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.2249898,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCGTTTGGCGC-1,GSE215120,3603,1012,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,3.6081044,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCTCATGTCCC-1,GSE215120,14482,2882,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.0357685,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1


In [142]:
#exclude any samples with <100 cells
table(MEL$integration_id)
#exclude AM2, AM4, CM2
MEL <- subset(MEL, !(subset = integration_id %in% c("GSE215120_Acral_MEL_AM2","GSE215120_Acral_MEL_AM4","GSE215120_Cut_MEL_CM2")))
table(MEL$integration_id)


GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM2 GSE215120_Acral_MEL_AM3 
                    260                      23                     101 
GSE215120_Acral_MEL_AM4 GSE215120_Acral_MEL_AM5 GSE215120_Acral_MEL_AM6 
                      9                     279                     115 
  GSE215120_Cut_MEL_CM1   GSE215120_Cut_MEL_CM2   GSE215120_Cut_MEL_CM3 
                    133                      32                     100 
 GSE215120_MEL_mets_CM1 
                    162 


GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM3 GSE215120_Acral_MEL_AM5 
                    260                     101                     279 
GSE215120_Acral_MEL_AM6   GSE215120_Cut_MEL_CM1   GSE215120_Cut_MEL_CM3 
                    115                     133                     100 
 GSE215120_MEL_mets_CM1 
                    162 

In [143]:
#join layers and then split them by integration_id
Layers(MEL[["RNA"]])
#join layers
MEL[["RNA"]] <- JoinLayers(MEL[["RNA"]])
Layers(MEL[["RNA"]])
#split layers
MEL[["RNA"]] <- split(MEL[["RNA"]], f = MEL$integration_id)
Layers(MEL[["RNA"]])


[1] "counts.1.1"   "counts.1.2"   "counts.3.1"   "counts.3.2"   "counts.4.3"  
 [6] "counts.5.1"   "counts.6.1"   "data.1.1"     "data.3.1"     "data.5.1"    
[11] "data.6.1"     "scale.data.1" "data.1.2"     "data.3.2"     "scale.data.2"
[16] "data.4.3"     "scale.data.3"

[1] "scale.data" "data"       "counts"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "counts.GSE215120_Acral_MEL_AM1" "counts.GSE215120_Acral_MEL_AM3"
 [3] "counts.GSE215120_Acral_MEL_AM5" "counts.GSE215120_Acral_MEL_AM6"
 [5] "counts.GSE215120_Cut_MEL_CM1"   "counts.GSE215120_Cut_MEL_CM3"  
 [7] "counts.GSE215120_MEL_mets_CM1"  "scale.data"                    
 [9] "data.GSE215120_Acral_MEL_AM1"   "data.GSE215120_Acral_MEL_AM3"  
[11] "data.GSE215120_Acral_MEL_AM5"   "data.GSE215120_Acral_MEL_AM6"  
[13] "data.GSE215120_Cut_MEL_CM1"     "data.GSE215120_Cut_MEL_CM3"    
[15] "data.GSE215120_MEL_mets_CM1"

In [144]:
#record number of cells
MEL
MEL@project.name
head(MEL@meta.data)
tail(MEL@meta.data)
table(MEL$integration_id)

An object of class Seurat 
33538 features across 1150 samples within 1 assay 
Active assay: RNA (33538 features, 2000 variable features)
 15 layers present: counts.GSE215120_Acral_MEL_AM1, counts.GSE215120_Acral_MEL_AM3, counts.GSE215120_Acral_MEL_AM5, counts.GSE215120_Acral_MEL_AM6, counts.GSE215120_Cut_MEL_CM1, counts.GSE215120_Cut_MEL_CM3, counts.GSE215120_MEL_mets_CM1, scale.data, data.GSE215120_Acral_MEL_AM1, data.GSE215120_Acral_MEL_AM3, data.GSE215120_Acral_MEL_AM5, data.GSE215120_Acral_MEL_AM6, data.GSE215120_Cut_MEL_CM1, data.GSE215120_Cut_MEL_CM3, data.GSE215120_MEL_mets_CM1

[1] "GSE215120"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE215120_AM1_AAACCTGGTTGCTCCT-1,GSE215120,20298,3789,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,0.9754656,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGATGTCCAAATGC-1,GSE215120,5574,1721,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,6.0459275,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGTAGTCGGTGTTA-1,GSE215120,13432,2759,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.1515783,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCCAGAGCCAA-1,GSE215120,17143,2659,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.2249898,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCGTTTGGCGC-1,GSE215120,3603,1012,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,3.6081044,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCTCATGTCCC-1,GSE215120,14482,2882,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.0357685,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1


,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE215120_CM1_mets_TTGGTTTAGTGCAACG-1,GSE215120,627,355,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,3.668262,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTAGTCAGTTCTCTT-1,GSE215120,2740,748,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,4.197080,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTCACACACCGGAAA-1,GSE215120,1174,529,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,22.146508,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTCATGTCCACTAGA-1,GSE215120,732,395,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,2.322404,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTGGAGGTTGAGAGC-1,GSE215120,4265,1376,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,4.220399,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTGGTTTCGAGGCAA-1,GSE215120,671,384,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,1.043219,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1



GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM3 GSE215120_Acral_MEL_AM5 
                    260                     101                     279 
GSE215120_Acral_MEL_AM6   GSE215120_Cut_MEL_CM1   GSE215120_Cut_MEL_CM3 
                    115                     133                     100 
 GSE215120_MEL_mets_CM1 
                    162 

In [145]:
#re-export seurat object ready for integration
saveRDS(MEL, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/GSE215120_myeloid_int.RDS")

In [146]:
#remove all objects in R
rm(list = ls())

## PRJNA907381

In [147]:
MEL <- readRDS("/scratch/user/s4436039/scdata/Myeloid_Cells/PRJNA907381_myeloid.RDS")

In [148]:
MEL
MEL@project.name
head(MEL@meta.data)

An object of class Seurat 
36601 features across 2723 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)
 17 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, counts.7, counts.8, data.1, data.2, data.3, data.4, data.5, data.6, data.7, data.8, scale.data
 2 dimensional reductions calculated: pca, umap

[1] "PRJNA907381"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
PRJNA907381_MEL022_iLN_AAAGAACCAGCGCGTT-1,PRJNA907381,17285,3704,LN mets,Metastatic Melanoma,MEL022,PRJNA907381_MEL022_iLN,2.100087,6,6
PRJNA907381_MEL022_iLN_AAAGGGCTCCATAGAC-1,PRJNA907381,42925,6544,LN mets,Metastatic Melanoma,MEL022,PRJNA907381_MEL022_iLN,3.934770,6,6
PRJNA907381_MEL022_iLN_AACAAAGCAAGTATAG-1,PRJNA907381,16549,3569,LN mets,Metastatic Melanoma,MEL022,PRJNA907381_MEL022_iLN,6.689226,6,6
PRJNA907381_MEL022_iLN_AACAAGACAGGATTCT-1,PRJNA907381,18108,3854,LN mets,Metastatic Melanoma,MEL022,PRJNA907381_MEL022_iLN,4.870775,6,6
PRJNA907381_MEL022_iLN_AACCACATCTTTCCAA-1,PRJNA907381,31754,5097,LN mets,Metastatic Melanoma,MEL022,PRJNA907381_MEL022_iLN,3.231089,6,6
PRJNA907381_MEL022_iLN_AACCATGAGAAGTCTA-1,PRJNA907381,26158,4705,LN mets,Metastatic Melanoma,MEL022,PRJNA907381_MEL022_iLN,4.908632,6,6


In [149]:
table(MEL$sample_type)
table(MEL$cancer_type)
table(MEL$patient_id)
table(MEL$sample_id)


      LN mets uninvolved LN 
         1536          1187 


            Healthy Metastatic Melanoma 
               1187                1536 


MEL002 MEL009 MEL014 MEL018 MEL022 
   614    404    743    785    177 


PRJNA907381_MEL002_iLN PRJNA907381_MEL002_uLN PRJNA907381_MEL009_iLN 
                   164                    450                    404 
PRJNA907381_MEL014_iLN PRJNA907381_MEL014_uLN PRJNA907381_MEL018_iLN 
                   422                    321                    369 
PRJNA907381_MEL018_uLN PRJNA907381_MEL022_iLN 
                   416                    177 

In [150]:
#split by cancer_type
MEL_Tu <- subset(MEL, subset = cancer_type %in% c("Metastatic Melanoma"))
MEL_H <- subset(MEL, subset = cancer_type %in% c("Healthy"))

#set site and sample_type_major metadata
MEL_Tu@meta.data$site <- "lymph node"
MEL_H@meta.data$site <- "lymph node"

MEL_Tu@meta.data$sample_type_major <- "metastatic tumour"
MEL_H@meta.data$sample_type_major <- "healthy"

#set subtype metadata
MEL_Tu@meta.data$cancer_subtype <- "Melanoma"
MEL_H@meta.data$cancer_subtype <- "NA"

#Merge seurat objects back together
MEL <- merge(MEL_Tu, y = c(MEL_H), project = "PRJNA907381")

## Up to here

In [ ]:
#set integration_id metadata
MEL@meta.data$integration_id <- MEL@meta.data$sample_id

In [ ]:
MEL
MEL@project.name
head(MEL@meta.data)

An object of class Seurat 
33538 features across 1214 samples within 1 assay 
Active assay: RNA (33538 features, 2000 variable features)
 23 layers present: counts.1.1, counts.1.2, counts.2.1, counts.2.2, counts.3.1, counts.3.2, counts.4.1, counts.4.3, counts.5.1, counts.6.1, data.1.1, data.2.1, data.3.1, data.4.1, data.5.1, data.6.1, scale.data.1, data.1.2, data.2.2, data.3.2, scale.data.2, data.4.3, scale.data.3

[1] "GSE215120"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE215120_AM1_AAACCTGGTTGCTCCT-1,GSE215120,20298,3789,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,0.9754656,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGATGTCCAAATGC-1,GSE215120,5574,1721,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,6.0459275,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGTAGTCGGTGTTA-1,GSE215120,13432,2759,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.1515783,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCCAGAGCCAA-1,GSE215120,17143,2659,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.2249898,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCGTTTGGCGC-1,GSE215120,3603,1012,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,3.6081044,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCTCATGTCCC-1,GSE215120,14482,2882,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.0357685,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1


In [ ]:
#exclude any samples with <100 cells
table(MEL$integration_id)
#exclude AM2, AM4, CM2
MEL <- subset(MEL, !(subset = integration_id %in% c("GSE215120_Acral_MEL_AM2","GSE215120_Acral_MEL_AM4","GSE215120_Cut_MEL_CM2")))
table(MEL$integration_id)


GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM2 GSE215120_Acral_MEL_AM3 
                    260                      23                     101 
GSE215120_Acral_MEL_AM4 GSE215120_Acral_MEL_AM5 GSE215120_Acral_MEL_AM6 
                      9                     279                     115 
  GSE215120_Cut_MEL_CM1   GSE215120_Cut_MEL_CM2   GSE215120_Cut_MEL_CM3 
                    133                      32                     100 
 GSE215120_MEL_mets_CM1 
                    162 


GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM3 GSE215120_Acral_MEL_AM5 
                    260                     101                     279 
GSE215120_Acral_MEL_AM6   GSE215120_Cut_MEL_CM1   GSE215120_Cut_MEL_CM3 
                    115                     133                     100 
 GSE215120_MEL_mets_CM1 
                    162 

In [ ]:
#join layers and then split them by integration_id
Layers(MEL[["RNA"]])
#join layers
MEL[["RNA"]] <- JoinLayers(MEL[["RNA"]])
Layers(MEL[["RNA"]])
#split layers
MEL[["RNA"]] <- split(MEL[["RNA"]], f = MEL$integration_id)
Layers(MEL[["RNA"]])


[1] "counts.1.1"   "counts.1.2"   "counts.3.1"   "counts.3.2"   "counts.4.3"  
 [6] "counts.5.1"   "counts.6.1"   "data.1.1"     "data.3.1"     "data.5.1"    
[11] "data.6.1"     "scale.data.1" "data.1.2"     "data.3.2"     "scale.data.2"
[16] "data.4.3"     "scale.data.3"

[1] "scale.data" "data"       "counts"

Splitting ‘counts’, ‘data’ layers. Not splitting ‘scale.data’. If you would like to split other layers, set in `layers` argument.



[1] "counts.GSE215120_Acral_MEL_AM1" "counts.GSE215120_Acral_MEL_AM3"
 [3] "counts.GSE215120_Acral_MEL_AM5" "counts.GSE215120_Acral_MEL_AM6"
 [5] "counts.GSE215120_Cut_MEL_CM1"   "counts.GSE215120_Cut_MEL_CM3"  
 [7] "counts.GSE215120_MEL_mets_CM1"  "scale.data"                    
 [9] "data.GSE215120_Acral_MEL_AM1"   "data.GSE215120_Acral_MEL_AM3"  
[11] "data.GSE215120_Acral_MEL_AM5"   "data.GSE215120_Acral_MEL_AM6"  
[13] "data.GSE215120_Cut_MEL_CM1"     "data.GSE215120_Cut_MEL_CM3"    
[15] "data.GSE215120_MEL_mets_CM1"

In [ ]:
#record number of cells
MEL
MEL@project.name
head(MEL@meta.data)
tail(MEL@meta.data)
table(MEL$integration_id)

An object of class Seurat 
33538 features across 1150 samples within 1 assay 
Active assay: RNA (33538 features, 2000 variable features)
 15 layers present: counts.GSE215120_Acral_MEL_AM1, counts.GSE215120_Acral_MEL_AM3, counts.GSE215120_Acral_MEL_AM5, counts.GSE215120_Acral_MEL_AM6, counts.GSE215120_Cut_MEL_CM1, counts.GSE215120_Cut_MEL_CM3, counts.GSE215120_MEL_mets_CM1, scale.data, data.GSE215120_Acral_MEL_AM1, data.GSE215120_Acral_MEL_AM3, data.GSE215120_Acral_MEL_AM5, data.GSE215120_Acral_MEL_AM6, data.GSE215120_Cut_MEL_CM1, data.GSE215120_Cut_MEL_CM3, data.GSE215120_MEL_mets_CM1

[1] "GSE215120"

,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE215120_AM1_AAACCTGGTTGCTCCT-1,GSE215120,20298,3789,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,0.9754656,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGATGTCCAAATGC-1,GSE215120,5574,1721,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,6.0459275,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAAGTAGTCGGTGTTA-1,GSE215120,13432,2759,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.1515783,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCCAGAGCCAA-1,GSE215120,17143,2659,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.2249898,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCGTTTGGCGC-1,GSE215120,3603,1012,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,3.6081044,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1
GSE215120_AM1_AAATGCCTCATGTCCC-1,GSE215120,14482,2882,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.0357685,12,12,skin,primary tumour,Acral Melanoma,GSE215120_Acral_MEL_AM1


,orig.ident,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,RNA_snn_res.0.2,seurat_clusters,site,sample_type_major,cancer_subtype,integration_id
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSE215120_CM1_mets_TTGGTTTAGTGCAACG-1,GSE215120,627,355,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,3.668262,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTAGTCAGTTCTCTT-1,GSE215120,2740,748,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,4.197080,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTCACACACCGGAAA-1,GSE215120,1174,529,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,22.146508,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTCATGTCCACTAGA-1,GSE215120,732,395,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,2.322404,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTGGAGGTTGAGAGC-1,GSE215120,4265,1376,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,4.220399,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1
GSE215120_CM1_mets_TTTGGTTTCGAGGCAA-1,GSE215120,671,384,LN metastasis,Cutaneous Melanoma,CM1,GSE215120_MEL_mets_CM1,1.043219,11,11,lymph node,metastatic tumour,Melanoma,GSE215120_MEL_mets_CM1



GSE215120_Acral_MEL_AM1 GSE215120_Acral_MEL_AM3 GSE215120_Acral_MEL_AM5 
                    260                     101                     279 
GSE215120_Acral_MEL_AM6   GSE215120_Cut_MEL_CM1   GSE215120_Cut_MEL_CM3 
                    115                     133                     100 
 GSE215120_MEL_mets_CM1 
                    162 

In [ ]:
#re-export seurat object ready for integration
saveRDS(MEL, "/scratch/user/s4436039/scdata/Myeloid_Cells/Myeloid_Cells_Integrate/GSE215120_myeloid_int.RDS")

In [ ]:
#remove all objects in R
rm(list = ls())